In [ ]:
import glob
from openai import OpenAI
import gradio as gr

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

context = {}

employees = glob.glob('knowledge-base/employees/*')

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc=''
    with open(employee, 'r', encoding='utf-8') as f:
        doc= f.read()
    context[name]=doc

def get_context(message):
    relevant_context=[] 
    for title, details in context.items():
        if title.lower() in message.lower():
            relevant_context.append('\n\nAdditional context might help you to answer the questions:\n\n')
            relevant_context.append(details)
    if relevant_context==[]:
        relevant_context.append(message)

    return str(relevant_context) 

system_message = 'You are an expert in answering accurate questions about Insurellm, the Insurance Tech Company.Give brief, accurate answers.'\
'If you don"t know the answer, say so. Do not make anything up if you haven"t been provided with relevant context.'       

def chat(message, history):
    message = get_context(message)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model="llama3.2", messages=messages, stream=True)

    response=''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

gr.ChatInterface(fn=chat, type='messages').launch()        
